In [1]:
import torch as th

In [2]:
from torch import optim,nn

In [3]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad = True)
target = th.tensor([[1.],[1],[0],[0]],requires_grad = True)

In [4]:
model = nn.Linear(2,1)
opt = optim.SGD(params=model.parameters(), lr = 0.1)
model.parameters()

<generator object Module.parameters at 0x000001840233AED0>

In [5]:
def trainings(iterations):
    for training in range(iterations):
        opt.zero_grad()
        pred = model(data)
        loss = ((pred-target)**2).sum()
        loss.backward()
        opt.step()
        print(loss.data)
trainings(20)
    

tensor(0.5374)
tensor(0.3690)
tensor(0.2788)
tensor(0.2129)
tensor(0.1627)
tensor(0.1243)
tensor(0.0950)
tensor(0.0727)
tensor(0.0555)
tensor(0.0425)
tensor(0.0325)
tensor(0.0248)
tensor(0.0190)
tensor(0.0145)
tensor(0.0111)
tensor(0.0085)
tensor(0.0065)
tensor(0.0050)
tensor(0.0038)
tensor(0.0029)


In [6]:
model(data[0])

tensor([0.9738], grad_fn=<AddBackward0>)

# Federated Learning

In [7]:
import syft as sy
hook = sy.TorchHook(th)
naufil = sy.VirtualWorker(hook,id="naufil")
faizan = sy.VirtualWorker(hook,id="faizan")

Keras (Tensorflow) not available.


In [8]:
data_naufil = data[0:2].send(naufil)
target_naufil = target[0:2].send(naufil)

In [9]:
naufil._objects

{73130792627: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 83807204976: tensor([[1.],
         [1.]], requires_grad=True)}

In [10]:
data_faizan = data[2:4].send(faizan)
target_faizan = target[2:4].send(faizan)

In [11]:
faizan._objects

{25242060783: tensor([[1., 0.],
         [0., 0.]], requires_grad=True), 3805664151: tensor([[0.],
         [0.]], requires_grad=True)}

In [12]:
datasets = [(data_naufil,target_naufil),(data_faizan,target_faizan)]

In [13]:
datasets

[((Wrapper)>[PointerTensor | me:42007985342 -> naufil:73130792627],
  (Wrapper)>[PointerTensor | me:82227322542 -> naufil:83807204976]),
 ((Wrapper)>[PointerTensor | me:19621806101 -> faizan:25242060783],
  (Wrapper)>[PointerTensor | me:54864530328 -> faizan:3805664151])]

In [14]:
def trainings(iterations=20):
    model = nn.Linear(2,1)
    opt = optim.SGD(params = model.parameters(),lr = 0.1)
    for i in range(iterations):
        for _data,_target in datasets:
            model = model.send(_data.location)
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()
            
            model = model.get()
            
            print(loss.get())
            
trainings(20)    

tensor(0.2672, requires_grad=True)
tensor(0.5164, requires_grad=True)
tensor(0.2304, requires_grad=True)
tensor(0.3727, requires_grad=True)
tensor(0.1735, requires_grad=True)
tensor(0.2723, requires_grad=True)
tensor(0.1298, requires_grad=True)
tensor(0.1998, requires_grad=True)
tensor(0.0972, requires_grad=True)
tensor(0.1470, requires_grad=True)
tensor(0.0729, requires_grad=True)
tensor(0.1085, requires_grad=True)
tensor(0.0546, requires_grad=True)
tensor(0.0802, requires_grad=True)
tensor(0.0410, requires_grad=True)
tensor(0.0594, requires_grad=True)
tensor(0.0308, requires_grad=True)
tensor(0.0441, requires_grad=True)
tensor(0.0231, requires_grad=True)
tensor(0.0328, requires_grad=True)
tensor(0.0173, requires_grad=True)
tensor(0.0244, requires_grad=True)
tensor(0.0130, requires_grad=True)
tensor(0.0182, requires_grad=True)
tensor(0.0098, requires_grad=True)
tensor(0.0135, requires_grad=True)
tensor(0.0073, requires_grad=True)
tensor(0.0101, requires_grad=True)
tensor(0.0055, requi